In [1]:
import numpy as np
import os
import random
import math
import heapq

In [2]:
def sign(x):
    return 1 if x > 0 else -1 if x < 0 else 0

def orientation(a, b, c):
    cross = (b[0] - a[0]) * (c[1] - a[1]) - (b[1] - a[1]) * (c[0] - a[0])  # 外積
    return sign(cross) # a, b, c の順番の向き(1: 反時計周り, -1: 反時計回り, 0: 一直線)

def segments_intersect(p1, p2, q1, q2):
    if p1==q1 or p1==q2 or p2==q1 or p2==q2:
        return False
    if (max(p1[0], p2[0]) < min(q1[0], q2[0]) or
        max(q1[0], q2[0]) < min(p1[0], p2[0]) or
        max(p1[1], p2[1]) < min(q1[1], q2[1]) or
        max(q1[1], q2[1]) < min(p1[1], p2[1])):
        return False
    o1 = orientation(p1, p2, q1)
    o2 = orientation(p1, p2, q2)
    o3 = orientation(q1, q2, p1)
    o4 = orientation(q1, q2, p2)
    return (o1 * o2 <= 0) and (o3 * o4 <= 0)

In [3]:
def parse_input(filename):
    with open(os.path.join('in', filename), 'r') as file:
        n, m, k = list(map(int, file.readline().strip().split()))
        
        d_pos = []
        for _ in range(n):
            x, y = list(map(int, file.readline().strip().split()))
            d_pos.append((x, y))
        
        s_pos = []
        for _ in range(m):
            x, y = list(map(int, file.readline().strip().split()))
            s_pos.append((x, y))
        
        prob = []
        for _ in range(k):
            p = list(map(float, file.readline().strip().split()))
            prob.append(p)
    return (n, m, k, d_pos, s_pos, prob)

n, m, k, d_pos, s_pos, prob = parse_input('0001.txt')
(n, m, k, len(d_pos), len(s_pos), len(prob))

(10, 348, 29, 10, 348, 29)

In [4]:
def sorter(in_prob, i):
    p = np.array(prob[i])
    out_prob1 = in_prob * p
    out_prob2 = in_prob * (1-p)
    return (out_prob1, out_prob2)

in_prob = np.ones(n)
sorter(in_prob, 0)

(array([0.6403, 0.8234, 0.112 , 0.7991, 0.8508, 0.7999, 0.8654, 0.4284,
        0.707 , 0.5603]),
 array([0.3597, 0.1766, 0.888 , 0.2009, 0.1492, 0.2001, 0.1346, 0.5716,
        0.293 , 0.4397]))

In [5]:
def opt_processor(out_prob):
    max_i = np.argmax(out_prob)
    max_prob = out_prob[max_i]
    
    return (max_i, max_prob)

def calc_opt_score(out_probs):
    probs = [0.0 for _ in range(n)]
    to_processors = [0 for _ in range(n)]
    for out_prob in out_probs:
        (max_i, max_prob) = opt_processor(out_prob)
        probs[max_i] += max_prob
        to_processors[max_i] += 1
    score = calc_score(probs)

    return (score, probs, to_processors)

def calc_score(probs):
    return round(1_000_000_000 * (n - sum(probs))/n)

out_probs = sorter(in_prob, 0)
score, probs, to_processors = calc_opt_score(out_probs)
print('score:', format(int(score), ','))
print('probs:', probs)
print('to_processors:', to_processors)

score: 824,660,000
probs: [0.0, 0.0, 0.888, 0.0, 0.0, 0.0, 0.8654, 0.0, 0.0, 0.0]
to_processors: [0, 0, 1, 0, 0, 0, 1, 0, 0, 0]


## 1階層

In [6]:
opt_i = 0
opt_score = 1_000_000_000
opt_probs = [0.0 for _ in range(n)]
opt_to_processors = [0 for _ in range(n)]

for i in range(k):
    out_probs = sorter(in_prob, i)
    score, probs, to_processors = calc_opt_score(out_probs)
    if opt_score > score:
        print(f"improve: {opt_i} => {i}, {format(int(opt_score), ',')} => {format(int(score), ',')}")
        print('probs:', probs)
        opt_i = i
        opt_score = score
        opt_probs = probs
        opt_to_processors = to_processors

improve: 0 => 0, 1,000,000,000 => 824,660,000
probs: [0.0, 0.0, 0.888, 0.0, 0.0, 0.0, 0.8654, 0.0, 0.0, 0.0]
improve: 0 => 5, 824,660,000 => 821,960,000
probs: [0.0, 0.0, 0.8861, 0.0, 0.0, 0.0, 0.8943, 0.0, 0.0, 0.0]


## 2階層

In [7]:
in_prob = np.ones(n)
in_probs = sorter(in_prob, 0)
out_probs = []
for in_prob in in_probs:
    ps = sorter(in_prob, 0)
    out_probs.extend(ps)
score, probs, to_processors = calc_opt_score(out_probs)
print('score:', format(int(score), ','))
print('probs:', probs)
print('to_processors:', to_processors)

score: 796,981,102
probs: [0.0, 0.0, 0.788544, 0.0, 0.0, 0.0, 0.7489171599999999, 0.0, 0.0, 0.49272782]
to_processors: [0, 0, 1, 0, 0, 0, 1, 0, 0, 2]


## 山登り

In [8]:
in_prob = np.ones(n)
out_probs = [in_prob]
score, probs, to_processors = calc_opt_score(out_probs)
print(f"score: {format(int(score), ',')}, prob: {np.mean(probs):.2f}")

def opt_sorter(in_prob):
    opt_i = 0
    opt_prob = 0.0
    for i in range(k):
        out_probs = sorter(in_prob, i)
        sum_prob = 0.0
        for out_prob in out_probs:
            (max_i, max_prob) = opt_processor(out_prob)
            sum_prob += max_prob
        if opt_prob < sum_prob:
            opt_prob = sum_prob
            opt_i = i
    return (opt_i, opt_prob)

random.seed(0)
for i in range(101):
    rand_i = random.randint(0, len(out_probs)-1)
    in_prob = out_probs.pop(rand_i)
    (opt_i, opt_prob) = opt_sorter(in_prob)
    ps = sorter(in_prob, opt_i)
    out_probs.extend(ps)
    score, probs, to_processors = calc_opt_score(out_probs)
    if i%10==0:
        print(f"i: {i}, score: {format(int(score), ',')}, prob: {np.mean(probs):.2f}")

score: 900,000,000, prob: 0.10
i: 0, score: 821,960,000, prob: 0.18
i: 10, score: 655,903,763, prob: 0.34
i: 20, score: 617,878,564, prob: 0.38
i: 30, score: 610,466,690, prob: 0.39
i: 40, score: 572,437,912, prob: 0.43
i: 50, score: 558,250,662, prob: 0.44
i: 60, score: 553,140,627, prob: 0.45
i: 70, score: 528,034,733, prob: 0.47
i: 80, score: 527,921,177, prob: 0.47
i: 90, score: 515,337,503, prob: 0.48
i: 100, score: 513,428,246, prob: 0.49


In [16]:
in_prob = np.ones(n)
out_probs = [in_prob]
score, probs, to_processors = calc_opt_score(out_probs)
print(f"score: {format(int(score), ',')}, prob: {np.mean(probs):.2f}")

def opt_sorter2(in_prob):
    opt_i = 0
    opt_prob = 0.0
    opt_processors = [0, 0]
    for i in range(k):
        out_probs = sorter(in_prob, i)
        sum_prob = 0.0
        processors = []
        for out_prob in out_probs:
            (max_i, max_prob) = opt_processor(out_prob)
            sum_prob += max_prob
            processors.append(max_i)
        if opt_prob < sum_prob:
            opt_prob = sum_prob
            opt_i = i
            opt_processors = processors
    return (opt_i, opt_prob, opt_processors)

sorters = []
processors = []
for i in range(6):
    in_prob = out_probs.pop(0)
    (opt_i, opt_prob, opt_processors) = opt_sorter2(in_prob)
    ps = sorter(in_prob, opt_i)
    out_probs.extend(ps)
    sorters.append(opt_i)
    if len(processors) > 0:
        processors.pop(0)
    processors.extend(opt_processors)
    score, probs, to_processors = calc_opt_score(out_probs)
    if i%10==0:
        print(f"i: {i}, score: {format(int(score), ',')}, prob: {np.mean(probs):.2f}")
print(f"i: {i}, score: {format(int(score), ',')}, prob: {np.mean(probs):.2f}")
print(f'sorters: {sorters}')
print(f'processors: {processors}')

score: 900,000,000, prob: 0.10
i: 0, score: 821,960,000, prob: 0.18
i: 5, score: 619,331,449, prob: 0.38
sorters: [5, 13, 11, 16, 11, 0]
processors: [8, 0, 1, 4, 6, 4, 2]


In [17]:
sum_prob = 0.0
processor_cnt = {i: 0 for i in range(n)}
processor_probs = []
for i, p in enumerate(processors):
    sum_prob += out_probs[i][p]
    processor_cnt[p] += 1
    processor_probs.append((out_probs[i][p], i, p))
print(sum_prob / n)
print(f'processor_cnt: {processor_cnt}')
processor_probs.sort(reverse=True)
print(f'processor_probs: {processor_probs}')

0.3806685508018
processor_cnt: {0: 1, 1: 1, 2: 1, 3: 0, 4: 2, 5: 0, 6: 1, 7: 0, 8: 1, 9: 0}
processor_probs: [(0.6991037682419999, 4, 6), (0.6976272388800001, 6, 2), (0.64635844, 0, 8), (0.5253906138180001, 1, 0), (0.47318080991999995, 2, 1), (0.42029957311200006, 5, 4), (0.344725064046, 3, 4)]


## 配置方法
- 入口から近い順に sorter を配置
- 接続可能な sorter を見つけて接続
- 割合が高い sorter に対して、近い processor の配置場所に processor を配置、接続
- 接続ができない場合は、接続可能な processor に接続

In [18]:
# 基準点
entrance = (0, 5000)

# 距離を計算する関数
def calculate_distance(point):
    return math.sqrt((point[0] - entrance[0])**2 + (point[1] - entrance[1])**2)

# 2分木関連
def parent(i):
    if i==0:
        return None
    return (i-1)//2

def left(i):
    return 2*i+1

def right(i):
    return 2*i+2

def calc_dist(pos1, pos2):
    return math.sqrt((pos1[0] - pos2[0])**2 + (pos1[1] - pos2[1])**2)

def connectable(p1, p2):
    for i in range(len(edges)):
        if edges[i] is None:
            continue
        (q1, q2) = edges[i]
        if segments_intersect(p1, p2, q1, q2):
            return False
    return True

'''
# ヒープ (優先度付きキュー) を作成
que = []
for (i, pos) in enumerate(s_pos):
    distance = calculate_distance(pos)
    heapq.heappush(que, (distance, i))

# 近い順に取得
ans_d = [None for _ in range(n)]
ans_s = [[-1] for _ in range(m)]
# 最初の配置
(_, i) = heapq.heappop(que)
ans_start = n+i
now_i = 0
ans_s[i] = [sorters[now_i]]
xy = s_pos[i]
edges = [((0, 5000), xy)]
done_cnt = 1
indexes = [i]

# sorter の配置
while True:
    # 左の子
    (_, i) = heapq.heappop(que)
    while not connectable(xy, s_pos[i]):
        (_, i) = heapq.heappop(que)
    v1 = i
    # 接続
    ans_s[v1] = [sorters[left(now_i)]]
    ans_s[indexes[now_i]].append(n+v1)
    edges.append((xy, s_pos[v1]))
    indexes.append(v1)
    print(f'now_i: {now_i}, index: {indexes[now_i]}, left: {left(now_i)}, sorter: {sorters[left(now_i)]}, index: {indexes[left(now_i)]}')
    done_cnt += 1
    if done_cnt == len(sorters):
        break
    
    # 右の子
    (_, i) = heapq.heappop(que)
    while not connectable(xy, s_pos[i]):
        (_, i) = heapq.heappop(que)
    v2 = i
    # 接続
    ans_s[v2] = [sorters[right(now_i)]]
    ans_s[indexes[now_i]].append(n+v2)
    edges.append((xy, s_pos[v2]))
    indexes.append(v2)
    print(f'now_i: {now_i}, index: {indexes[now_i]}, right: {right(now_i)}, sorter: {sorters[right(now_i)]}, index: {indexes[right(now_i)]}')
    done_cnt += 1
    if done_cnt == len(sorters):
        break
    
    now_i += 1
    xy = s_pos[indexes[now_i]]
    
# processor の接続
for (_, i, p) in processor_probs:
    base_i = len(sorters)
    now_i = base_i + i
    parent_i = parent(now_i)
    si = indexes[parent_i]
    print(f'now_i: {now_i}, parent_i: {parent_i}, si: {si}, p: {p}')
    while len(ans_s[si]) < 3:
        ans_s[si].append(-1)
    if left(parent_i) == now_i:
        child_i = 1
    else:
        child_i = 2
    
    di = -1
    xy = s_pos[indexes[parent_i]]
    if p in ans_d:
        ii = ans_d.index(p)
        if connectable(xy, d_pos[ii]):
            di = ii
    else:
        que = []
        for (i, pos) in enumerate(d_pos):
            if ans_d[i] is not None:
                continue
            dist = calc_dist(xy, pos)
            heapq.heappush(que, (dist, i))
        (dd, i) = heapq.heappop(que)
        while not connectable(xy, d_pos[i]):
            if len(que)==0:
                di = None
                break
            (dd, i) = heapq.heappop(que)
        di = i
    
    if di != -1 and connectable(xy, d_pos[di]):
        ans_s[si][child_i] = di
        ans_d[di] = p
        edges.append((xy, d_pos[di]))
'''

"\n# ヒープ (優先度付きキュー) を作成\nque = []\nfor (i, pos) in enumerate(s_pos):\n    distance = calculate_distance(pos)\n    heapq.heappush(que, (distance, i))\n\n# 近い順に取得\nans_d = [None for _ in range(n)]\nans_s = [[-1] for _ in range(m)]\n# 最初の配置\n(_, i) = heapq.heappop(que)\nans_start = n+i\nnow_i = 0\nans_s[i] = [sorters[now_i]]\nxy = s_pos[i]\nedges = [((0, 5000), xy)]\ndone_cnt = 1\nindexes = [i]\n\n# sorter の配置\nwhile True:\n    # 左の子\n    (_, i) = heapq.heappop(que)\n    while not connectable(xy, s_pos[i]):\n        (_, i) = heapq.heappop(que)\n    v1 = i\n    # 接続\n    ans_s[v1] = [sorters[left(now_i)]]\n    ans_s[indexes[now_i]].append(n+v1)\n    edges.append((xy, s_pos[v1]))\n    indexes.append(v1)\n    print(f'now_i: {now_i}, index: {indexes[now_i]}, left: {left(now_i)}, sorter: {sorters[left(now_i)]}, index: {indexes[left(now_i)]}')\n    done_cnt += 1\n    if done_cnt == len(sorters):\n        break\n    \n    # 右の子\n    (_, i) = heapq.heappop(que)\n    while not connectable(xy, s_po

In [77]:
pos_data = []
for (i, pos) in enumerate(s_pos):
    distance = calculate_distance(pos)
    pos_data.append((distance, i))
pos_data.sort()

def dfs(node_i, pos_i):
    if node_i == 0:
        frm = (0, 5000)
    else:
        frm = node_pos[parent(node_i)]
    print(node_i, len(edges), len(sorters))
    if node_i < len(sorters):  # 分別器の配置
        start_i = pos_i
        for (ii, (_, i)) in enumerate(pos_data[start_i:]):
            to = s_pos[i]
            if not connectable(frm, to):
                continue
            ans_s[i] = [sorters[node_i]]
            edges[node_i] = (frm, to)
            node_pos[node_i] = to
            indexes[node_i] = i
            if node_i == 0:
                ans_start[0] = n+i
            else:
                parent_i = indexes[parent(node_i)]
                ans_s[parent_i].append(n+i)
            if dfs(node_i+1, start_i+ii+1):
                return True
            else:
                ans_s[parent_i].pop()
    else:  # 処理装置の配置
        # 接続したい装置が未配置なら、一番近いところから順に確認して、接続可能なところに配置して、接続する。接続済みの場所は不可
        # すでに配置済みなら、接続できるか確認する。できなければ、接続できるところに接続する
        p = processors[node_i-len(sorters)]
        if p in ans_d:  # 配置済みの場合
            ii = ans_d.index(p)
            to = d_pos[ii]
            if connectable(frm, to):
                edges[node_i] = (frm, to)
                node_pos[node_i] = to
                parent_i = indexes[parent(node_i)]
                ans_s[parent_i].append(ii)
                if len(ans_s[parent_i]) > 3:
                       print(f'1 parent_i: {parent_i}, node_i: {node_i}, ans_s: {ans_s[parent_i]}')
                if node_i+1 == len(sorters)+len(processors):
                    return True
                elif dfs(node_i+1, -1):
                    return True
                else:
                    ans_s[parent_i].pop()
            else:
                # 接続できるところに接続する
                d_dist = []
                for (i, pos) in enumerate(d_pos):
                    dist = calc_dist(frm, pos)
                    d_dist.append((dist, i))
                d_dist.sort()
                for (_, ii) in d_dist:
                    to = d_pos[ii]
                    if connectable(frm, to):
                        edges[node_i] = (frm, to)
                        node_pos[node_i] = to
                        parent_i = indexes[parent(node_i)]
                        ans_s[parent_i].append(ii)
                        if len(ans_s[parent_i]) > 3:
                               print(f'2 parent_i: {parent_i}, node_i: {node_i}, ans_s: {ans_s[parent_i]}')
                        if node_i+1 == len(sorters)+len(processors):
                            return True
                        elif dfs(node_i+1, -1):
                            return True
                        else:
                            ans_s[parent_i].pop()
        else:  # 未配置の場合
            d_dist = []
            for (i, pos) in enumerate(d_pos):
                if ans_d[i] is not None:
                    continue
                dist = calc_dist(frm, pos)
                d_dist.append((dist, i))
            d_dist.sort()
            for (_, ii) in d_dist:
                to = d_pos[ii]
                if connectable(frm, to):
                    ans_d[ii] = p
                    edges[node_i] = (frm, to)
                    node_pos[node_i] = to
                    parent_i = indexes[parent(node_i)]
                    ans_s[parent_i].append(ii)
                    if len(ans_s[parent_i]) > 3:
                           print(f'3 parent_i: {parent_i}, node_i: {node_i}, ans_s: {ans_s[parent_i]}')
                    if node_i+1 == len(sorters)+len(processors):
                        return True
                    elif dfs(node_i+1, -1):
                        return True
                    else:
                        ans_s[parent_i].pop()
                        ans_d[ii] = None
    return False  # 最後まで接続できたものなし

    
# sorters, processors
ans_d = [None for _ in range(n)]
ans_s = [[-1] for _ in range(m)]
ans_start = [-1]
indexes = [-1 for _ in range(m)]
edges = [None for _ in range(len(sorters)+len(processors))]
node_pos = [None for _ in range(len(sorters)+len(processors))]
dfs(0, 0)

0 13 6
1 13 6
2 13 6
3 13 6
4 13 6
5 13 6
6 13 6
7 13 6
8 13 6
9 13 6
9 13 6
10 13 6
11 13 6
12 13 6
11 13 6
12 13 6
11 13 6
12 13 6
10 13 6
11 13 6
12 13 6
12 13 6
11 13 6
12 13 6


True

In [78]:
not_exists = []
for i in range(n):
    if i not in ans_d:
        not_exists.append(i)
for (i, di) in enumerate(ans_d):
    if di is None:
        ans_d[i] = not_exists.pop()

for (i, a) in enumerate(ans_s):
    if ans_s[i][0] == -1:
        continue
    if ans_s[i][1] == -1 and ans_s[i][2] == -1:
        xy = s_pos[i]
        for (ii, pos) in enumerate(d_pos):
            if connectable(xy, pos):
                ans_s[i][1] = ii
                ans_s[i][2] = ii
                break
    elif ans_s[i][1] == -1:
        ans_s[i][1] = ans_s[i][2]
    elif ans_s[i][2] == -1:
        ans_s[i][2] = ans_s[i][1]

In [79]:
sorters, len(sorters), processors, len(processors), indexes[:len(processors)], len(indexes)

([5, 13, 11, 16, 11, 0],
 6,
 [8, 0, 1, 4, 6, 4, 2],
 7,
 [267, 329, 180, 192, 334, 97, -1],
 348)

In [80]:
ans_s[97]

[0, 0, 2]

In [81]:
print(*ans_d)
print(ans_start[0])
for a in ans_s:
    print(*a)

8 4 2 0 9 1 7 6 5 3
277
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
0 0 2
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
11 107 0
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
16 3 5
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
5 339 190
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1